# Chapter 05. Logistic Classification

- Reminder
- Computing Hypothesis
- Computing Cost Function
- Evaluation
- Higher Implementation

## Reminder: Logistic Regression

P(x=1) = 1-P(x=0)

2-dim size의 vector가 주어졌을 때, 0 또는 1 중에 어떤 쪽에 더 가까운지 구하게 된다.

|x · m| = (m, d) x (d, 1) = (m, 1)

[0, 1, 0..] - binary classification 문제의 setting

### Hypothesis

$$ H(X) = \frac{1}{1+e^{-W^T X}} $$

### Cost

$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$

If $y \simeq H(x)$, cost is near 0.

If $y \neq H(x)$, cost is high.


$W$을 이용하여 0과 1에 근사하도록 함.

H(x) = P(x=1; w) = 1-p(x=0; w)

### Weight Updata via Gradient Descent

이를 통해 cost function을 훈련시킬 수 있다.

$$ W := W - \alpha \frac{\partial}{\partial W} cost(W) $$
$\alpha$: Learning rate

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

### Training Data

In [31]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

|x_data| = (6, 2)

|y_data| = (6,)

m=6, d=2

In [32]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

```torch.Tensor``` format으로 만든다.

In [69]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


---

## Computing the Hypothesis

$$ H(X) = \frac{1}{1+e^{-W^T X}} $$

=> $ H(X) = \frac{1}{1+e^{-X·W}} $

=> $ H(X) = \frac{1}{1+e^{-X·W + b}} $

### torch.exp()

PyTorch has a ```torch.exp()``` function that resembles the exponential function.

In [70]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


d = 2

In [71]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [72]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

```x_train.matmul(W)``` - $X·W$ = ```torch.matmal(X, W)```

In [73]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


### sigmoid()

In [74]:
print('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equals:  tensor([0.7311])


In [75]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [76]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


---

## Computing the Cost Function  (Low-Level)

$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$

```hypothesis```와 ```y_train```의 차이를 알고자 한다(difference)

In [77]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


### Loss of One Element

In [78]:
-(y_train[0] * torch.log(hypothesis[0]) + 
  (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward>)

hypothesis[0]: scalar = logP(x=1; w)

1-hypothesis[0]: logP(x=0; w) = 1-logP(x=1; w)

### Loss of Entire Batch

In [79]:
losses = -(y_train * torch.log(hypothesis) + 
           (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [80]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


### ```F.binary_cross_entropy``` pytorch simple function

In [81]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

위에서 시행했던 코드를 ```F.binary_cross_entropy``` 함수를 이용하여 한 번에 표현 가능함.

---

## Training Procedure Code

### low level

In [93]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [83]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1) # Optimization 수행

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + 
             (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad() # Zero Gradient 해 주기
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


### ```F.binary_cross_entropy```

In [96]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031672
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


---

## Evaluation

모델 학습이 끝난 후, training set과 우리의 model이 얼마나 잘 맞는지 확인하기.

### hypothesis

In [84]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


hypothesis[:5]로 다섯 개만 출력하였으니 값은 다섯 개만 출력됨.

```hypothesis```을 ```binary prediection```으로 바꿈. (0.5와 비교한 것을 기준으로)
    
- range of hypothesis: [0, 1] R

- range of binary predictions: 0 or 1

### binary prediction

In [85]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


### compare with y_train

In [86]:
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [87]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


### accuracy

In [88]:
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

The model has an accuracy of 100.00% for the training set.


---

## Higher Implementation with Class

```nn.Module``` 사용

### logistic regression 행하는 class 선언

In [98]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [99]:
model = BinaryClassifier()

In [100]:
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.548766 Accuracy 83.33%
Epoch   10/100 Cost: 0.821105 Accuracy 66.67%
Epoch   20/100 Cost: 0.577588 Accuracy 83.33%
Epoch   30/100 Cost: 0.483939 Accuracy 83.33%
Epoch   40/100 Cost: 0.398343 Accuracy 83.33%
Epoch   50/100 Cost: 0.317014 Accuracy 83.33%
Epoch   60/100 Cost: 0.243594 Accuracy 83.33%
Epoch   70/100 Cost: 0.187757 Accuracy 100.00%
Epoch   80/100 Cost: 0.157877 Accuracy 100.00%
Epoch   90/100 Cost: 0.144273 Accuracy 100.00%
Epoch  100/100 Cost: 0.134425 Accuracy 100.00%
